In [1]:
!cd /Users/lc/Developer/kuzu &&  make clean-python-api && make release NUM_THREADS=20 

rm -rf tools/python_api/build
cd external && \
	mkdir -p build && \
	cd build && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../ && \
	cmake --build . --config Release -- -j 20
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/external/build
[100%] Built target apache_arrow
mkdir -p build/release && \
	cd build/release && \
	cmake    -DCMAKE_BUILD_TYPE=Release ../.. && \
	cmake --build . --config Release -- -j 20
-- pybind11 v2.11.0 dev1
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/lc/Developer/kuzu/build/release
[  0%] Built target kuzu_transaction
[  0%] Built target utf8proc
[  0%] Built target kuzu_common_csv_reader
[  1%] Built target kuzu_binder_expression
[  2%] Built target kuzu_common_arrow
[  2%] Built target kuzu_catalog
[  3%] Built target kuzu_common_vector
[  3%] Built target kuzu_common_data_chunk
[  3%] Built target kuzu_binder
[  4%] Built target kuzu_common_task_system
[  5%] Built tar

In [2]:
import sys
sys.path.append('/Users/lc/Developer/kuzu/tools/python_api/build')
import kuzu

In [3]:
import torch_geometric.transforms as T
from torch_geometric.datasets import OGB_MAG


dataset = OGB_MAG(root='/Users/lc/Developer/ogb_pytorch', preprocess='metapath2vec', transform=T.ToUndirected())
data = dataset[0]

In [4]:
import torch

In [5]:
DB_PATH = '/Users/lc/Developer/kuzu_ogb_mag'
db = kuzu.Database(DB_PATH)

In [6]:
conn = kuzu.Connection(db)

In [7]:
fs, gs = db.get_torch_geometric_remote_backend()

In [8]:
data

HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 128] },
  institution={ x=[8740, 128] },
  field_of_study={ x=[59965, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)

In [7]:
from torch_geometric.loader import NeighborLoader


In [10]:
loader_kuzu = NeighborLoader(
    data=(fs, gs),
    num_neighbors={key.edge_type : [30] * 2 for key in gs.get_all_edge_attrs()},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None,
    num_workers=10,
)

In [11]:
import time
times = []
for i in range(10):
    start = time.time()
    next(iter(loader_kuzu))
    end = time.time()
    if i > 0:
        times.append(end - start)
        print("Total time", end - start)
print("=====")
print(sum(times) / len(times))

Total time 3.2379000186920166
Total time 2.9140219688415527
Total time 2.865478992462158
Total time 2.644171953201294
Total time 2.554901123046875
Total time 2.5458102226257324
Total time 2.395853042602539
Total time 2.2535243034362793
Total time 2.485074043273926
=====
2.6551928520202637


In [12]:
next(iter(loader_kuzu))

HeteroData(
  paper={
    num_nodes=36213,
    id=[36213],
    x=[36213, 128],
    year=[36213],
    y=[36213],
    input_id=[128],
    batch_size=128
  },
  institution={
    num_nodes=343,
    id=[343],
    x=[343, 128]
  },
  author={
    num_nodes=5737,
    id=[5737],
    x=[5737, 128]
  },
  field_of_study={
    num_nodes=2797,
    id=[2797],
    x=[2797, 128]
  },
  (paper, has_topic, field_of_study)={ edge_index=[2, 20795] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 960] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 12436] },
  (paper, cites, paper)={ edge_index=[2, 21998] },
  (author, writes, paper)={ edge_index=[2, 7985] },
  (paper, rev_writes, author)={ edge_index=[2, 9236] }
)

In [12]:
next(iter(loader_kuzu))

HeteroData(
  paper={
    num_nodes=35999,
    id=[35999],
    x=[35999, 128],
    year=[35999],
    y=[35999],
    input_id=[128],
    batch_size=128
  },
  institution={
    num_nodes=333,
    id=[333],
    x=[333, 128]
  },
  author={
    num_nodes=5709,
    id=[5709],
    x=[5709, 128]
  },
  field_of_study={
    num_nodes=2792,
    id=[2792],
    x=[2792, 128]
  },
  (paper, has_topic, field_of_study)={ edge_index=[2, 20795] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 942] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 12417] },
  (paper, cites, paper)={ edge_index=[2, 22022] },
  (author, writes, paper)={ edge_index=[2, 7942] },
  (paper, rev_writes, author)={ edge_index=[2, 9075] }
)

In [10]:
loader_pyg = NeighborLoader(
    data=data,
    num_neighbors={key: [30] * 2 for key in data.edge_types},
    # Use a batch size of 128 for sampling training nodes of type paper
    batch_size=128,
    input_nodes=('paper', torch.arange(0, data['paper'].num_nodes)),
    input_time=None,
    num_workers=10,
)

In [11]:
import time
times = []
for i in range(10):
    start = time.time()
    next(iter(loader_pyg))
    end = time.time()
    if i > 0:
        times.append(end - start)
        print(end - start)
print("=====")
print(sum(times) / len(times))

2.314298152923584
2.3115360736846924
2.287034034729004
2.268874168395996
2.238197088241577
2.3284809589385986
2.2262158393859863
2.288803815841675
2.3746721744537354
=====
2.2931235896216497


In [12]:
next(iter(loader_pyg))

HeteroData(
  paper={
    x=[36241, 128],
    year=[36241],
    y=[36241],
    train_mask=[36241],
    val_mask=[36241],
    test_mask=[36241],
    input_id=[128],
    batch_size=128
  },
  author={ x=[5721, 128] },
  institution={ x=[339, 128] },
  field_of_study={ x=[2798, 128] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 7957] },
  (paper, cites, paper)={ edge_index=[2, 22078] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 20795] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 957] },
  (paper, rev_writes, author)={ edge_index=[2, 9150] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 12424] }
)